In [1]:
import zinv.modules

Welcome to JupyROOT 6.18/00


In [2]:
help(zinv.modules.analyse)

Help on function analyse in module zinv.modules.analyse:

analyse(dataset_cfg, sequence_cfg, event_selection_cfg, physics_object_cfg, trigger_cfg, hdf_cfg, name='zinv', outdir='output', tempdir='_ccsp_temp', mode='multiprocessing', batch_opts='-q hep.q', ncores=0, nblocks_per_dataset=-1, nblocks_per_process=-1, nfiles_per_dataset=-1, nfiles_per_process=1, blocksize=1000000, cachesize=8, quiet=False, sample=None)



In [ ]:
zinv.modules.analyse(
    "configs/datasets.yaml",
    "configs/module_sequence.yaml",
    "configs/event_selection.yaml",
    "configs/object_selection.yaml",
    "configs/trigger_selection.yaml",
    "configs/hdf_output.yaml",
    outdir="results",
    tempdir="/vols/cms/sdb15/ZinvWidth/tables/_ccsp_temp/",
    mode="sge",
    batch_opts="-q hep.q -l h_rt=3:0:0 -l h_vmem=24G",
    #mode="multiprocessing",
    #ncores=0,
    nblocks_per_process=1,
    blocksize=1_000_000,
)

2019-08-11 11:08:49,102 - pysge.area - INFO - Creating paths in /vols/cms/sdb15/ZinvWidth/tables/_ccsp_temp/tpd_20190811_110846_fl8pv2w_


2019-08-11 11:35:45,942 - pysge.submitter - INFO - Submitted 9294156.1-4112:1


2019-08-11 11:36:18,249 - pysge.monitor - INFO - Resubmitting 9294156.14: /vols/cms/sdb15/ZinvWidth/tables/_ccsp_temp/tpd_20190811_110846_fl8pv2w_/task_00013
